# Chapter 05

1️⃣ 학습 모델 저장하고 재사용하기

In [5]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter = ',', unpack = True, dtype = 'float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

# 신경망 모델 구성
global_step = tf.Variable(0, trainable = True, name = 'global_step')    # 모델을 저장할 때 쓸 변수 만들기. 학습에 직접 사용되지는 않고 학습 횟수를 카운팅

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2,10],-1.,1.))
L1 = tf.nn.relu(tf.matmul(X,W1))

W2 = tf.Variable(tf.random_uniform([10,20],-1.,1.))
L2 = tf.nn.relu(tf.matmul(L1,W2))

W3 = tf.Variable(tf.random_uniform([20,3],-1.,1.))
model = tf.matmul(L2,W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels =Y, logits = model))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost,global_step = global_step)   # 최적화 함수가 학습용 변수들을 최적화할 때마다 global_step변수의 값을 1씩 증가시킴

# 신경망 모델 학습
sess= tf.Session()
saver = tf.train.Saver(tf.global_variables())   # global_variables는 앞서 정의한 변수들을 가져오는 함수. 정의한 변수들을 모두 가져와서 이 변수들을 파일에 저장하거나 이전에 학습한 결과를 불러와 담는 변수들로 사용함

ckpt = tf.train.get_checkpoint_state('./model')   #'./model' 디렉터리에 기존에 학습해둔 모델이 있는지 확인해서 모 델이 있으면 그 값들을 불러오고 아니면 변수를 새로 초기화
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    
else:
    sess.run(tf.global_variables_initializer())
    

for step in range(2):
    sess.run(train_op, feed_dict = {X:x_data, Y:y_data})
    
    print('Step:%d,'%sess.run(global_step), 'Cost:%.3f' % sess.run(cost,feed_dict = {X:x_data, Y:y_data}))
    
saver.save(sess, './model/dnn.ckpt', global_step = global_step)   # 최적화가 끝난 뒤 학습된 변수들을 지정한 체크포인트 파일에 저장

# 결과 확인
prediction = tf.argmax(model,1)
target = tf.argmax(Y,1)
print('예측값: ', sess.run(prediction, feed_dict={X:x_data}))
print('실제값: ', sess.run(target, feed_dict={Y:y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

Step:1, Cost:1.281
Step:2, Cost:1.081
예측값:  [0 1 0 0 0 1]
실제값:  [0 1 2 0 0 2]
정확도: 66.67


In [7]:
ckpt = tf.train.get_checkpoint_state('./model')   #'./model' 디렉터리에 기존에 학습해둔 모델이 있는지 확인해서 모 델이 있으면 그 값들을 불러오고 아니면 변수를 새로 초기화
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    
else:
    sess.run(tf.global_variables_initializer())
    

for step in range(2):
    sess.run(train_op, feed_dict = {X:x_data, Y:y_data})
    
    print('Step:%d,'%sess.run(global_step), 'Cost:%.3f' % sess.run(cost,feed_dict = {X:x_data, Y:y_data}))
    
saver.save(sess, './model/dnn.ckpt', global_step = global_step)   # 최적화가 끝난 뒤 학습된 변수들을 지정한 체크포인트 파일에 저장

# 결과 확인
prediction = tf.argmax(model,1)
target = tf.argmax(Y,1)
print('예측값: ', sess.run(prediction, feed_dict={X:x_data}))
print('실제값: ', sess.run(target, feed_dict={Y:y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

INFO:tensorflow:Restoring parameters from ./model\dnn.ckpt-2
Step:3, Cost:0.920
Step:4, Cost:0.804
예측값:  [0 1 2 0 0 2]
실제값:  [0 1 2 0 0 2]
정확도: 100.00


2️⃣ 텐서보드 사용하기

In [8]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter = ',', unpack = True, dtype = 'float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

# 신경망 모델 구성
global_step = tf.Variable(0,trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):    # 신경망의 각 계층에 다음 코드를 덧붙여줌. with tf.name_scope로 묶은 블록은 텐서보드에서 한 계층 내부를 표현
    W1 = tf.Variable(tf.random_uniform([2,10],-1.,1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X,W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10,20],-1.,1.), name = 'W2')
    L2 = tf.nn.relu(tf.matmul(L1,W2))
    
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20,3], -1.,1.), name = 'W3')
    model = tf.matmul(L2,W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
    train_op = optimizer.minimize(cost,global_step = global_step)
    tf.summary.scalar('cost', cost)   # 손실값을 추적하기 위해 수집할 값을 지정하는 코드
    
# 신경망 모델 학습
sess=tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model_2')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    
else:
    sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()   # tf.summary.merge_all함수를 통해 앞서 지정한 텐서들을 수집
writer = tf.summary.FileWriter('./logs', sess.graph)    # tf.summary.FileWriter함수를 이용해 그래프와 텐서들의 값을 저장할 디렉터리를 설정

for step in range(100):   # 최적화 실행
    sess.run(train_op, feed_dict = {X:x_data, Y:y_data})
    print('Step: %d, '%sess.run(global_step), 'Cost: %.3f' %sess.run(cost,feed_dict = {X:x_data, Y:y_data}))
    summary = sess.run(merged,feed_dict= {X:x_data, Y:y_data})   # 앞서 merged로 모아둔 텐서의 값들을 계산하여 수집
    writer.add_summary(summary,global_step = sess.run(global_step))   # writer.add_summary 함수 이용해 해당 값들을 앞서 지정한 디렉토리에 저장
    
saver.save(sess,'./model_2/dnn.ckpt',global_step = global_step)

# 결과확인
prediction = tf.argmax(model,1)
target = tf.argmax(Y,1)
print('예측값: ', sess.run(prediction, feed_dict={X:x_data}))
print('실제값: ', sess.run(target, feed_dict={Y:y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

Step: 1,  Cost: 1.818
Step: 2,  Cost: 1.702
Step: 3,  Cost: 1.592
Step: 4,  Cost: 1.489
Step: 5,  Cost: 1.393
Step: 6,  Cost: 1.305
Step: 7,  Cost: 1.224
Step: 8,  Cost: 1.152
Step: 9,  Cost: 1.087
Step: 10,  Cost: 1.030
Step: 11,  Cost: 0.980
Step: 12,  Cost: 0.937
Step: 13,  Cost: 0.901
Step: 14,  Cost: 0.871
Step: 15,  Cost: 0.847
Step: 16,  Cost: 0.826
Step: 17,  Cost: 0.808
Step: 18,  Cost: 0.794
Step: 19,  Cost: 0.782
Step: 20,  Cost: 0.773
Step: 21,  Cost: 0.764
Step: 22,  Cost: 0.755
Step: 23,  Cost: 0.747
Step: 24,  Cost: 0.739
Step: 25,  Cost: 0.731
Step: 26,  Cost: 0.724
Step: 27,  Cost: 0.717
Step: 28,  Cost: 0.711
Step: 29,  Cost: 0.704
Step: 30,  Cost: 0.697
Step: 31,  Cost: 0.689
Step: 32,  Cost: 0.680
Step: 33,  Cost: 0.670
Step: 34,  Cost: 0.660
Step: 35,  Cost: 0.650
Step: 36,  Cost: 0.640
Step: 37,  Cost: 0.631
Step: 38,  Cost: 0.622
Step: 39,  Cost: 0.613
Step: 40,  Cost: 0.606
Step: 41,  Cost: 0.598
Step: 42,  Cost: 0.592
Step: 43,  Cost: 0.586
Step: 44,  Cost: 0.5